In [20]:
!pip install aesara > /dev/null
!pip install pymc > /dev/null
!pip install arviz > /dev/null
!pip install statsmodels > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.12.0 which is incompatible.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.0 which is incompatible.


In [21]:
!pip install --upgrade --force-reinstall numpy scipy scikit-learn > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aesara 2.9.4 requires numpy<2.0.0,>=1.17.0, but you have numpy 2.3.1 which is incompatible.
aesara 2.9.4 requires scipy<=1.12.0,>=0.14, but you have scipy 1.16.0 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.1 which is incompatible.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.0 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.1 which is incompatible.
plotnine 0.14.6 requires scipy<1.16.0,>=1.8.0, but you have scipy 1.16.0 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.3.1 which is incompatible.


In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import seaborn as sns
import arviz as az

import pymc as pm
import pytensor.tensor as at

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [23]:
X, y = make_classification(n_samples=1000, n_features=2, n_informative=2, n_redundant=0,
                           n_clusters_per_class=1, flip_y=0.1, class_sep=2.0, random_state=42)

X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape, y_train.shape)


(700, 2) (700,)


In [24]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def log_prior(w, sigma_prior=5.0):
    return -0.5 * np.sum((w / sigma_prior)**2, axis=1)

def log_likelihood(w, X, y):
    logits = X @ w.T
    probs = sigmoid(logits)
    y = y[:, None]
    ll = y * np.log(probs + 1e-8) + (1 - y) * np.log(1 - probs + 1e-8)
    return np.sum(ll, axis=0)

def log_posterior(w, X, y):
    return log_prior(w) + log_likelihood(w, X, y)


In [25]:
def rbf_kernel(X, h=-1):
    N, D = X.shape
    sq_dists = np.sum((X[:, None, :] - X[None, :, :])**2, axis=2)
    if h <= 0:
        h = np.median(sq_dists)
        h = np.sqrt(0.5 * h / np.log(N + 1e-6))
    K = np.exp(-sq_dists / (2 * h**2))
    grad_K = -(X[:, None, :] - X[None, :, :]) * K[:, :, None] / (h**2)
    return K, grad_K


In [26]:
def svgd_step(particles, log_post_fn, X, y, stepsize=0.05):
    N, D = particles.shape
    eps = 1e-4
    scores = np.zeros_like(particles)
    for i in range(D):
        shift = np.zeros(D)
        shift[i] = eps
        forward = log_post_fn(particles + shift[None, :], X, y)
        backward = log_post_fn(particles - shift[None, :], X, y)
        scores[:, i] = (forward - backward) / (2 * eps)
    K, grad_K = rbf_kernel(particles)
    phi = (K @ scores) / N + np.sum(grad_K, axis=1) / N
    particles += stepsize * phi
    return particles


In [27]:
np.random.seed(42)
num_particles = 100
particles = np.random.randn(num_particles, 2) * 2.0

svgd_samples = []
start = time.time()
for it in range(500):
    particles = svgd_step(particles, log_posterior, X_train, y_train, stepsize=0.05)
    if it % 50 == 0:
        print(f"SVGD iter {it}")
    svgd_samples.append(particles.copy())
end = time.time()

svgd_time = end - start
print(f"SVGD time: {svgd_time:.2f}s")


SVGD iter 0
SVGD iter 50
SVGD iter 100
SVGD iter 150
SVGD iter 200
SVGD iter 250
SVGD iter 300
SVGD iter 350
SVGD iter 400
SVGD iter 450
SVGD time: 14.11s


In [28]:
with pm.Model() as logistic_model:
    w = pm.Normal("w", mu=0, sigma=5, shape=2)
    logits = pm.math.dot(X_train, w)
    y_obs = pm.Bernoulli("y_obs", logit_p=logits, observed=y_train)


In [29]:
with logistic_model:
    start = time.time()
    trace = pm.sample(draws=500, tune=0, chains=1, cores=1, target_accept=0.8, progressbar=True)
    end = time.time()
hmc_time = end - start
print(f"HMC time: {hmc_time:.2f}s")


Output()

HMC time: 1.64s


In [30]:
def compute_log_lik(X_test, y_test, weights_samples):
    log_liks = []
    for w in weights_samples:
        logits = X_test @ w
        p = sigmoid(logits)
        ll = y_test * np.log(p + 1e-8) + (1 - y_test) * np.log(1 - p + 1e-8)
        log_liks.append(np.mean(ll))
    return np.mean(log_liks), np.std(log_liks)


In [31]:
svgd_flat = np.vstack(svgd_samples[-100:])
svgd_mean_ll, svgd_std_ll = compute_log_lik(X_test, y_test, svgd_flat)
print(f"SVGD test log-likelihood: {svgd_mean_ll:.3f} ± {svgd_std_ll:.3f}")


SVGD test log-likelihood: -0.197 ± 0.001


In [32]:
hmc_samples = trace.posterior['w'].stack(draws=("chain", "draw")).values.T
hmc_mean_ll, hmc_std_ll = compute_log_lik(X_test, y_test, hmc_samples)
print(f"HMC test log-likelihood: {hmc_mean_ll:.3f} ± {hmc_std_ll:.3f}")


HMC test log-likelihood: -0.197 ± 0.003


In [33]:
ess_hmc = az.ess(trace)
print(ess_hmc)
ess_hmc_min = ess_hmc['w'].min().values
ess_per_sec_hmc = ess_hmc_min / hmc_time
print(f"HMC ESS/sec: {ess_per_sec_hmc:.2f}")


<xarray.Dataset> Size: 32B
Dimensions:  (w_dim_0: 2)
Coordinates:
  * w_dim_0  (w_dim_0) int64 16B 0 1
Data variables:
    w        (w_dim_0) float64 16B 434.4 206.0
HMC ESS/sec: 125.94


In [34]:
from statsmodels.tsa.stattools import acf

def autocorr_ess(chain):
    acf_vals = acf(chain, nlags=40, fft=True)
    act = 1 + 2 * np.sum(acf_vals[1:])
    return len(chain) / act

svgd_dim_ess = [autocorr_ess(svgd_flat[:, d]) for d in range(2)]
ess_svgd = min(svgd_dim_ess)
ess_per_sec_svgd = ess_svgd / svgd_time
print(f"SVGD ESS/sec: {ess_per_sec_svgd:.2f}")


SVGD ESS/sec: 918.05


In [35]:
results = pd.DataFrame({
    'Sampler': ['SVGD', 'HMC'],
    'ESS/sec': [ess_per_sec_svgd, ess_per_sec_hmc],
    'Test log-likelihood': [svgd_mean_ll, hmc_mean_ll]
})

print(results)


  Sampler     ESS/sec  Test log-likelihood
0    SVGD  918.045134            -0.196780
1     HMC  125.938361            -0.197307
